In [ ]:
from autoshop import all as autoshop
import pandas as pd

In [ ]:
empty_basket = False

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
driver = autoshop.chrome.driver()
autoshop.tesco.login(driver=driver)

In [ ]:
autoshop.tesco.make_changes_to_nth_order(driver=driver, n=0)

In [ ]:
# TODO: Need to add an assert here for date

In [ ]:
if empty_basket:
    autoshop.tesco.empty_basket(driver=driver)

In [ ]:
df_shop = autoshop.google.get_shop()
df_food_conversion = autoshop.google.get_food_conversion()
df_tesco_food_map_raw = autoshop.google.get_tesco_food_map()

In [ ]:
df_tesco_food_map = (
    df_tesco_food_map_raw.dropna(subset=["amount"])
    .reset_index(drop=True)
    .assign(
        unit=lambda x: x["unit"].where(x["unit"] != "pack", ""),
        amountGram=lambda x: x[["amount", "unit"]].apply(
            lambda row: autoshop.unit.parse_to_grams(
                amount=row["amount"],
                unit=row["unit"],
            ),
            axis=1,
        ),
        amountMilliliter=lambda x: x[["amount", "unit"]].apply(
            lambda row: autoshop.unit.parse_to_milliliters(
                amount=row["amount"],
                unit=row["unit"],
            ),
            axis=1,
        ),
    )
    .merge(
        df_food_conversion,
        how="left",
        on=["food", "unit"],
    )
    .assign(
        amountGram=lambda x: x["amountGram"].fillna(x["amount"] * x["toGram"]),
    )[
        # We will ignore anything in milliliters for now
        lambda x: x["amountMilliliter"].isna()
    ]
)

In [ ]:
# Should be empty
df_tesco_food_map[lambda x: x["amountGram"].isna() & x["amountMilliliter"].isna()]

In [ ]:
df_food_shop = (
    df_shop.merge(df_food_conversion, how="left", on=["food", "unit"])
    .assign(
        amountGram=lambda x: x["amount"].where(
            x["unit"] == "g", x["amount"] * x["toGram"]
        ),
        quantityUnderlying=lambda x: x["amount"].astype(str) + " " + x["unit"],
    )
    .groupby(["food", "nameFood"])
    .aggregate(
        amountGram=pd.NamedAgg(
            column="amountGram", aggfunc=lambda x: x.sum(skipna=False)
        ),
        quantityUnderlying=pd.NamedAgg(
            column="quantityUnderlying", aggfunc=lambda x: ", ".join(x)
        ),
    )
    .reset_index()
)

display(df_food_shop)

In [ ]:
assert not df_food_shop.isna().any().any(), "There is a nan!!"

In [ ]:
import time

for _, row in df_food_shop.iterrows():
    time.sleep(3)
    food = row["food"]
    quantity_underlying = row["quantityUnderlying"]
    amount_grams = row["amountGram"]
    autoshop.logger.debug(
        f"Working on {food=}, {quantity_underlying=}, {amount_grams=}"
    )

    df_tesco = df_tesco_food_map[lambda x: x["food"] == food].sort_values(
        "order", ascending=True
    )

    if not df_tesco.empty:
        link = df_tesco.iloc[0]["link"]
        amount_tesco_grams = df_tesco.iloc[0]["amountGram"]
        amount_tesco = int(round(amount_grams / amount_tesco_grams, 0))
        amount_tesco = 1 if amount_tesco == 0 else amount_tesco

        info = f"{food=}, {quantity_underlying=}, {amount_tesco=}, {amount_grams=}, {amount_tesco_grams=}, {link=}"
        autoshop.logger.debug(f"Tesco mapping found with {info}")

        autoshop.tesco.add_food_to_basket_with_retry(
            driver=driver,
            url=link,
            amount=amount_tesco,
            info=info,
        )
    else:
        autoshop.logger.warning(
            f"No tesco mapping found for {food=}, {quantity_underlying=}"
        )

In [ ]:
autoshop.tesco.checkout(driver=driver, to_confirm_changes=True)